In [1]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pandas

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -------------------- ------------------- 5.8/11.1 MB 29.3 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 28.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
   ---------------------- ----------------- 7.3/13.0 MB 37.6 MB/s eta 0:00:01
   ---------------------------------------- 13.0/13.0 MB 37.1 MB/s eta 0:00:00

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ----------------

In [11]:
pip install langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 24.4 MB/s eta 0:00:00

   ---------- -----------------------------  4/15 [marshmallow]
   --------------------- ------------------  8/15 [yarl]
   ----------------------------- ---------- 11/15 [pydantic-settings]
   ---------------------------------- ----- 13/15 [aiohttp]
   ---------------------------------- ----- 13/15 [aiohttp]
   ---------------------------------- ----- 13/15 [aiohttp]
   ------------------------------------- -- 14/15 [langchain-community]
   ------------------------------------- -- 14/15 [langchain-community]
   ------------------------------------- -- 14/15 [langchain-community]
   ------------------------------------- -- 14/15 [langchain-community]
   ------------------------------------- -- 14/15 [langchain-community]
   ------------------------------------- -- 14/15 [langchain-community]
   ------------------------------------- -- 14/15 

In [39]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text
import urllib.parse

load_dotenv()

# Replace with your actual credentials
user = "postgres"
password = os.getenv("password")
host = "localhost"
port = "5432"
db_name = "job_role"

# URL-encode the password
encoded_password = urllib.parse.quote_plus(password)

# SQLAlchemy connection string
engine = create_engine(f"postgresql+psycopg2://{user}:{encoded_password}@{host}:{port}/{db_name}")

with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM cars"))
    for row in result:
        print(row)

('Ford', 'Mustang', 1964)
('Mercedes', 'G-Wagon', 1979)
('Mercedes', 'Benz', 1926)
('BMW', 'M1', 1978)
('Toyota', 'Supra', 1978)
('Volvo', 'p1800', 1968)
('Toyota', 'Celica', 1975)
('Rolls Royce', 'Phantom', 2003)
('Rolls Royce', 'Ghost', 2009)
('Rolls Royce', 'Culinan', 2019)
('Honda', 'Civic', 2007)


In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

# Set your Groq API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"  # Groq uses OpenAI-compatible endpoint

llm = ChatOpenAI(
    model="llama3-70b-8192",   
    temperature=0,
)

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities import SQLDatabase
from langchain.prompts import SystemMessagePromptTemplate
from langchain.schema.messages import SystemMessage

db = SQLDatabase(engine)

# System Prompt
custom_prompt = SystemMessage(content="""
You are a funny car expert helping users explore a car database. 
You should convert their questions into correct SQL queries and return relevant answers.
If the query is ambiguous, ask for clarification.
Be concise and helpful.
""")

#Create the toolkit and agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    system_message=custom_prompt   
)

#Ask the user for input
query = input("Ask your car-related question: ")
print(agent_executor.run(query))




> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: carsI have the list of tables in the database, and I see that there is a table called "cars". This is likely to be the most relevant table for my query.

Action: sql_db_schema
Action Input: cars
CREATE TABLE cars (
	brand VARCHAR(255), 
	model VARCHAR(255), 
	year INTEGER
)

/*
3 rows from cars table:
brand	model	year
Ford	Mustang	1964
Mercedes	G-Wagon	1979
Mercedes	Benz	1926
*/I have the schema of the "cars" table, and I see that it has columns for "brand", "model", and "year". I can use this information to construct a query to find Rolls Royce cars in the database.

Action: sql_db_query_checker
Action Input: SELECT brand, model FROM cars WHERE brand = 'Rolls Royce' LIMIT 10;SELECT "brand", "model" FROM "cars" WHERE "brand" = 'Rolls Royce' LIMIT 10;Thought: My que

In [ ]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"



In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

# Set your Groq API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"  # Groq uses OpenAI-compatible endpoint

llm = ChatOpenAI(
    model="llama3-70b-8192",   
    temperature=0,
)


In [60]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "You are a professional resume writer.\n"
    "Given the job role and a description of an achievement, write 2 word technical bullet points.\n"
    "Role: {role}\n"
    "Achievement: {achievement}\n"
    "Resume Bullet:"
)

llm = llm

parser = StrOutputParser()

# Setting up the chain
bullet_chain = prompt | llm | parser

# input
input_data = {
    "role": "AI ML Intern",
    "achievement": [
        "Engineered document chunking strategy to optimize context window for large-document queries.",
        "Integrated LLaMA 3.2 into a RAG pipeline to improve document-based answer generation.",
        "Built a custom workflow for extracting business card data on a local server using DIFY.",
        "Improved sentiment classification accuracy from 68% to 89% using NLP fine-tuning.",
        "Reduced training time in CNNs for medical image classification by 40%.",
        "Boosted diagnostic accuracy to 92% using custom CNN architectures in TensorFlow.",
        ]
}

# run the chain
result = bullet_chain.invoke(input_data)

# Output
print("Generated Resume Bullet:\n", result)


Generated Resume Bullet:
 Here are the 2-word technical bullet points for each achievement:

* **Chunking Strategy**
* **LLaMA Integration**
* **Custom Workflow**
* **NLP Fine-tuning**
* **Training Optimization**
* **CNN Architectures**

These bullet points are concise, technical, and highlight the specific skills and technologies used to achieve the described accomplishments.


In [62]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "You are an AI assistant that extracts skills from job descriptions.\n"
    "Given the following job description, return two lists:\n"
    "1. Technical Skills\n"
    "2. Soft Skills\n\n"
    "Job Description:\n{jd}\n\n"
    "Respond only with the categorized skills."
)

llm = llm

parser = StrOutputParser()

# Setting up the chain
skill_extractor_chain  = prompt | llm | parser

# input
input_data = {
    "jd" : "We are looking for a Data Scientist to join our analytics team. Responsibilities include building predictive models, collaborating with cross-functional teams, communicating insights to stakeholders, and maintaining scalable ML pipelines. Must be proficient in Python, SQL, and cloud platforms like AWS or GCP. Experience with Docker, Git, and TensorFlow is a plus. Strong problem-solving skills, attention to detail, and the ability to work independently are essential."
}

# run the chain
result = skill_extractor_chain.invoke(input_data)

# Output
print("Extracted Skills:\n")
print(result)

Extracted Skills:

Here are the extracted skills:

**Technical Skills:**

* Python
* SQL
* AWS
* GCP
* Docker
* Git
* TensorFlow
* ML pipelines

**Soft Skills:**

* Problem-solving
* Attention to detail
* Ability to work independently
* Collaboration
* Communication
